In [1]:
import openai
import re
import time
import json

import numpy as np
import os
from tqdm import tqdm
from pprint import pprint
from tenacity import retry, stop_after_attempt, wait_chain, wait_fixed

In [3]:
@retry(wait=wait_chain(*[wait_fixed(3) for i in range(3)] +
                       [wait_fixed(5) for i in range(2)] +
                       [wait_fixed(10)]))
def completion_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)

In [4]:
f = open("./obqa_test_gpt3_knowledge.jsonl", "r")
obqa_test = [json.loads(line.strip()) for line in f.readlines()]
print("len csqa-dev:", len(obqa_test))
print(obqa_test[0].keys())
print(obqa_test[0]["question"]["choices"])

len csqa-dev: 500
dict_keys(['id', 'question', 'answerKey', 'knowledges'])
[{'text': 'make more phone calls', 'label': 'A'}, {'text': 'quit eating lunch out', 'label': 'B'}, {'text': 'buy less with monopoly money', 'label': 'C'}, {'text': 'have lunch with friends', 'label': 'D'}]


In [ ]:
prompt_original_direct = open('prompt_original_direct.txt').read()
print(prompt_original_direct)

In [ ]:
import time
davinci_response_list = [] 
acc = 0
total = 0
answer_id = ["a", "b", "c", "d"]
for mention_idx, mention in tqdm(enumerate(obqa_test), total=len(obqa_test)):
    answer_gold = mention["answerKey"].lower()
    # answer = mention["answerKey"]
    prompt_q  = prompt_original_direct + "\nQ: " + mention["question"]["stem"] + "\n"
    prompt_q += "Answer Choices:\n"
    for idx, c in enumerate(mention["question"]["choices"]):
        prompt_q += '(%c) %s\n' % (answer_id[idx], c["text"])
    prompt_q += "The answer is "
#     break
    while True:
        try:
            response = completion_with_backoff(model="davinci", 
                                                prompt=prompt_q, 
                                                temperature=0, 
                                                max_tokens=5,
                                            )
            break
        except:
            time.sleep(3)
    davinci_response_list.append(response)
    answer_pred = response["choices"][0]["text"].strip().split("(")
    answer_pred = answer_pred[1][0] if len(answer_pred) > 1 else "a"
    acc += 1 if answer_pred == answer_gold else 0
    total += 1
    if total%50 ==0:
        print('Total %d correct %d acc %.4f' % (total , acc , acc / total))
#         print("gold:", answer_gold)
#         print("pred:", answer_pred)
#         print("acc:", acc)
#         print("total:", total)
#         print(prompt_q)
#         print(response)
#         break

print('Total %d correct %d acc %.4f' % (total , acc , acc / total))

# Total 1221 correct 547 acc 0.4480

In [ ]:
prompt_original_with_knowledge_before = open('prompt_original_with_knowledge copy.txt').read()
print(prompt_original_with_knowledge_before)

In [ ]:
import time
davinci_response_list = [] 
acc = 0
total = 0
answer_id = ["a", "b", "c", "d"]
for mention_idx, mention in tqdm(enumerate(obqa_test), total=len(obqa_test)):
    answer_gold = mention["answerKey"].lower()
    Know = mention["knowledges"][0]
    # answer = mention["answerKey"]
    prompt_q  = prompt_original_with_knowledge_before + f"\n{Know}\n Q: " + mention["question"]["stem"] + "\n"
    prompt_q += "Answer Choices:\n"
    for idx, c in enumerate(mention["question"]["choices"]):
        prompt_q += '(%c) %s\n' % (answer_id[idx], c["text"])
    prompt_q += "The answer is "
    # prompt_q += "A:"
#     break
    while True:
        try:
            response = completion_with_backoff(model="davinci", 
                                                prompt=prompt_q, 
                                                temperature=0, 
                                                max_tokens=5,
                                            )
            break
        except:
            time.sleep(3)
    davinci_response_list.append(response)
    answer_pred = response["choices"][0]["text"].strip().split("(")
    answer_pred = answer_pred[1][0] if len(answer_pred) > 1 else "a"
    acc += 1 if answer_pred == answer_gold else 0
    total += 1
    if total%50 ==0:
        print('Total %d correct %d acc %.4f' % (total , acc , acc / total))
#         print("gold:", answer_gold)
#         print("pred:", answer_pred)
#         print("acc:", acc)
#         print("total:", total)
#         print(prompt_q)
#         print(response)
#         break

print('Total %d correct %d acc %.4f' % (total , acc , acc / total))

# Total 1221 correct 547 acc 0.4480

In [ ]:
prompt_original_simple_instruct = open('prompt_original_simple_instruct.txt').read()
print(prompt_original_simple_instruct)

In [ ]:
import time
davinci_response_list = [] 
acc = 0
total = 0
answer_id = ["a", "b", "c", "d"]
for mention_idx, mention in tqdm(enumerate(obqa_test), total=len(obqa_test)):
    answer_gold = mention["answerKey"].lower()
    # answer = mention["answerKey"]
    prompt_q  = prompt_original_simple_instruct + "\nQ: " + mention["question"]["stem"] + "\n"
    prompt_q += "Answer Choices:\n"
    for idx, c in enumerate(mention["question"]["choices"]):
        prompt_q += '(%c) %s\n' % (answer_id[idx], c["text"])
    # prompt_q += "The answer is "
    prompt_q += "A:"
    # print(prompt_q)
#     break
    while True:
        try:
            response = completion_with_backoff(model="davinci", 
                                                prompt=prompt_q, 
                                                temperature=0, 
                                                max_tokens=80,
                                            )
            break
        except:
            time.sleep(3)
    davinci_response_list.append(response)
    answer_pred = response["choices"][0]["text"].strip().split("(")
    answer_pred = answer_pred[1][0] if len(answer_pred) > 1 else "a"
    acc += 1 if answer_pred == answer_gold else 0
    total += 1
    if total%50 ==0:
        print('Total %d correct %d acc %.4f' % (total , acc , acc / total))
#         print("gold:", answer_gold)
#         print("pred:", answer_pred)
#         print("acc:", acc)
#         print("total:", total)
#         print(prompt_q)
#         print(response)
#         break

print('Total %d correct %d acc %.4f' % (total , acc , acc / total))

# Total 1221 correct 547 acc 0.4480

In [ ]:
prompt_original = open('prompt_original.txt').read()
print(prompt_original)

In [ ]:
import time
davinci_response_list = [] 
acc = 0
total = 0
answer_id = ["a", "b", "c", "d"]
for mention_idx, mention in tqdm(enumerate(obqa_test), total=len(obqa_test)):
    answer_gold = mention["answerKey"].lower()
    # answer = mention["answerKey"]
    prompt_q  = prompt_original + "\nQ: " + mention["question"]["stem"] + "\n"
    prompt_q += "Answer Choices:\n"
    for idx, c in enumerate(mention["question"]["choices"]):
        prompt_q += '(%c) %s\n' % (answer_id[idx], c["text"])
    # prompt_q += "The answer is "
    prompt_q += "A:"
#     break
    while True:
        try:
            response = completion_with_backoff(model="davinci", 
                                                prompt=prompt_q, 
                                                temperature=0, 
                                                max_tokens=80,
                                            )
            break
        except:
            time.sleep(3)
    davinci_response_list.append(response)
    answer_pred = response["choices"][0]["text"].strip().split("(")
    answer_pred = answer_pred[1][0] if len(answer_pred) > 1 else "a"
    acc += 1 if answer_pred == answer_gold else 0
    total += 1
    if total%50 ==0:
        print('Total %d correct %d acc %.4f' % (total , acc , acc / total))